# Qiskit Runtime

Qiskit Runtime is a new architecture offered by IBM Quantum that streamlines computations requiring many iterations. These experiments will execute significantly faster within this improved hybrid quantum/classical process.

Using Qiskit Runtime, for example, a research team at IBM Quantum was able to achieve 120x speed 
up in their lithium hydride simulation (link to come). 

Qiskit Runtime allows authorized users to upload their Qiskit quantum programs for themselves or 
others to use. A Qiskit quantum program, also called a Qiskit runtime program, is a piece of Python code that takes certain inputs, performs
quantum and maybe classical computation, and returns the processing results. The same or other
authorized users can then invoke these quantum programs by simply passing in the required input parameters.

In [1]:
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(project='qiskit-runtime')  # Change this to your provider.


If you don't have an IBM Quantum account, you can sign up for one on the [IBM Quantum](https://quantum-computing.ibm.com/) page.

## Listing programs <a name='listing_program'>

The `provider.runtime` object is an instance of the [`IBMRuntimeService`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService) class and serves as the main entry point to using the runtime service. It has three methods that can be used to find metadata of available programs:
- `pprint_programs()`: pretty prints metadata of all available programs
- `programs()`: returns a list of `RuntimeProgram` instances
- `program()`: returns a single `RuntimeProgram` instance

The metadata of a runtime program includes its ID, name, description, version, input parameters, return values, interim results, maximum execution time, and backend requirements. Maximum execution time is the maximum amount of time, in seconds, a program can run before being forcibly terminated.

To print the metadata of all available programs:

In [2]:
provider.runtime.pprint_programs()

qasm3-runner:
  Name: qasm3-runner
  Description: A runtime program that takes one or more circuits, converts them to OpenQASM3, compiles them, executes them, and optionally applies measurement error mitigation. This program can also take and execute one or more OpenQASM3 strings. Note that this program can only run on a backend that supports OpenQASM3.
sampler:
  Name: sampler
  Description: Sample distributions generated by given circuits executed on the target backend.
estimator:
  Name: estimator
  Description: Expectation value estimator. A runtime program that estimates the value of an observable for an input quantum circuit. This program is in beta mode and is only available to select accounts.
sample-expval:
  Name: sample-expval
  Description: A sample expectation value program.
vqe:
  Name: vqe
  Description: Variational Quantum Eigensolver (VQE) to find the minimal eigenvalue of a Hamiltonian.
circuit-runner:
  Name: circuit-runner
  Description: A runtime program that takes

To print the metadata of the program `sample-program`:

In [3]:
program = provider.runtime.program('sample-program')
print(program)

sample-program:
  Name: sample-program
  Description: A sample runtime program.
  Creation date: 2021-07-02T13:45:13Z
  Update date: 2021-07-02T13:45:13Z
  Max execution time: 300
  Input parameters:
    Properties:
        - iterations:
            Description: Number of iterations to run. Each iteration generates a runs a random circuit.
            Minimum: 0
            Type: integer
            Required: True
  Interim results:
    Properties:
        - counts:
            Description: Histogram data of the circuit result.
            Type: object
            Required: False
        - iteration:
            Description: Iteration number.
            Type: integer
            Required: False
  Returns:
    Description: A string that says 'All done!'.
    Type: string


As you can see from above, the program `sample-program` is a simple program that has only 1 input parameter `iterations`, which indicates how many iterations to run. For each iteration it generates and runs a random 5-qubit circuit and returns the counts as well as the iteration number as the interim results. When the program finishes, it returns the sentence `All done!`. This program can only run for 300 seconds (5 minutes), and requires a backend that has at least 5 qubits.

## Invoking a runtime program <a name='invoking_program'>

You can use the [`IBMRuntimeService.run()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.run) method to invoke a runtime program. This method takes the following parameters:

- `program_id`: ID of the program to run
- `inputs`: Program input parameters. These input values are passed to the runtime program.
- `options`: Runtime options. These options control the execution environment. Currently the only available option is `backend_name`, which is required.
- `callback`: Callback function to be invoked for any interim results. The callback function will receive 2 positional parameters: job ID and interim result.
- `result_decoder`: Optional class used to decode job result.

Before we run a quantum program, we may want to define a callback function that would process interim results, which are intermediate data provided by a program while its still running. 

As we saw earlier, the metadata of `sample-program` says that its interim results are the iteration number and the counts of the randomly generated circuit. Here we define a simple callback function that just prints these interim results:

In [4]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}")

The following example runs the `sample-program` program with 3 iterations on `ibmq_montreal` and waits for its result. You can also use a different backend that supports Qiskit Runtime:

In [5]:
backend = provider.get_backend('ibmq_montreal')
program_inputs = {
    'iterations': 3
}
options = {'backend_name': backend.name()}
job = provider.runtime.run(program_id="sample-program",
                           options=options,
                           inputs=program_inputs,
                           callback=interim_result_callback
                          )
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

job id: c618jdik2ih5ha3l6mog
interim result: {'iteration': 0, 'counts': {'00000': 31, '00001': 11, '10000': 10, '10001': 10, '10010': 19, '10011': 11, '10100': 4, '10101': 6, '10110': 11, '10111': 8, '11000': 16, '11001': 4, '11010': 45, '11011': 16, '11100': 8, '11101': 9, '11110': 18, '11111': 8, '00010': 104, '00011': 47, '00100': 5, '00101': 10, '00110': 25, '00111': 10, '01000': 60, '01001': 35, '01010': 260, '01011': 119, '01100': 13, '01101': 12, '01110': 39, '01111': 40}}
interim result: {'iteration': 1, 'counts': {'00000': 99, '00001': 64, '10000': 13, '10001': 12, '10010': 9, '10011': 4, '10100': 21, '10101': 89, '10110': 6, '10111': 19, '11000': 19, '11001': 9, '11010': 5, '11011': 5, '11100': 26, '11101': 61, '11110': 11, '11111': 17, '00010': 37, '00011': 23, '00100': 73, '00101': 13, '00110': 20, '00111': 3, '01000': 105, '01001': 83, '01010': 30, '01011': 26, '01100': 79, '01101': 11, '01110': 22, '01111': 10}}
interim result: {'iteration': 2, 'counts': {'00000': 30, '00

The `run()` method returns a [`RuntimeJob`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.RuntimeJob.html#qiskit.providers.ibmq.runtime.RuntimeJob) instace, which is similar to the `Job` instance returned by regular `backend.run()`. `RuntimeJob` supports the following methods:

- `status()`: Return job status.
- `result()`: Wait for the job to finish and return the final result.
- `cancel()`: Cancel the job.
- `wait_for_final_state()`: Wait for the job to finish.
- `stream_results()`: Stream interim results. This can be used to start streaming the interim results if a `callback` function was not passed to the `run()` method. This method can also be used to reconnect a lost websocket connection.
- `job_id()`: Return the job ID.
- `backend()`: Return the backend where the job is run.
- `logs()`: Return job logs.
- `error_message()`: Returns the reason if the job failed and `None` otherwise.

## Retrieving old jobs

You can use the [`IBMRuntimeService.job()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.job) method to retrieve a previously executed runtime job. Attributes of this [`RuntimeJob`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.RuntimeJob.html#qiskit.providers.ibmq.runtime.RuntimeJob) instace can tell you about the execution:

In [6]:
retrieved_job = provider.runtime.job(job.job_id())
print(f"Job {retrieved_job.job_id()} is an execution instance of runtime program {retrieved_job.program_id}.")
print(f"This job ran on backend {retrieved_job.backend()} and had input parameters {retrieved_job.inputs}")

Job c618jdik2ih5ha3l6mog is an execution instance of runtime program sample-program.
This job ran on backend ibmq_montreal and had input parameters {'iterations': 3}


Similarly, you can use [`IBMRuntimeService.jobs()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.jobs) to get a list of jobs. You can specify a limit on how many jobs to return. The default limit is 10:

In [7]:
retrieved_jobs = provider.runtime.jobs(limit=1)
for rjob in retrieved_jobs:
    print(rjob.job_id())

c618jdik2ih5ha3l6mog


## Deleting a job

You can use the [`IBMRuntimeService.delete_job()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.delete_job) method to delete a job. You can only delete your own jobs, and this action cannot be reversed. 

In [8]:
provider.runtime.delete_job(job.job_id())